<a href="https://colab.research.google.com/github/mohripan/Machine-Learning/blob/main/Similar_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving data.csv to data.csv
move data.csv to upload/data.csv


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00


In [5]:
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('upload/data.csv')

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Map tokens to their IDs
        #    (4) Pad or truncate the sentence to `max_length`
        #    (5) Create attention masks for [PAD] tokens
        encoded_sent = tokenizer.encode_plus(
            text=sent,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=512,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks


# Get the lists of sentences.
sentences = df['0'].values

# Run the function `preprocessing_for_bert` on the sentences
input_ids, attention_masks = preprocessing_for_bert(sentences)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [22]:
from transformers import BertModel, BertTokenizer
import torch
import gc
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset

# Load the BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Set batch size (you may need to adjust this depending on your available memory)
batch_size = 16  # Adjust based on your system's capability

device = 'cuda' if torch.cuda.is_available() else 'cpu'
bert_model = bert_model.to(device)  # Move the model to the device

# Tokenize and encode the text
def tokenize_and_encode(text):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=512,
        add_special_tokens=True,  # Add ['CLS'] and ['SEP']
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',  # Return PyTorch tensors
        truncation=True
    )
    return encoded_text["input_ids"], encoded_text["attention_mask"]

# Embed the text
def bert_embeddings(input_ids, attention_masks):
    with torch.no_grad():  # Deactivate gradients for the following block
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        outputs = bert_model(input_ids, attention_mask=attention_masks)
        cls_embeddings = outputs[0][:, 0, :]  # Get the [CLS] embeddings
    return cls_embeddings.cpu().numpy()  # Move embeddings to CPU and convert to numpy

# Split the text into chunks and get embeddings for each
def get_document_embeddings(text):
    text_chunks = [text[i:i+512] for i in range(0, len(text), 512)]
    document_embeddings = []
    for chunk in text_chunks:
        input_ids, attention_masks = tokenize_and_encode(chunk)
        chunk_embeddings = bert_embeddings(input_ids, attention_masks)
        document_embeddings.append(chunk_embeddings)
    return np.concatenate(document_embeddings, axis=0)

# Get the document embeddings for the entire dataset
df['embeddings'] = df['0'].apply(get_document_embeddings)

# Flatten the list of embeddings into a 2D array
embeddings = np.concatenate(df['embeddings'].values, axis=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
!pip install corextopic

In [26]:
!pip install cPickle

ERROR: Could not find a version that satisfies the requirement cPickle (from versions: none)
ERROR: No matching distribution found for cPickle


In [37]:
from corextopic import corextopic as ct
import pickle

words = [str(i) for i in range(768)]

# Initialize CorEx
topic_model = ct.Corex(n_hidden=20, words=words, max_iter=200, verbose=False, seed=42)

# Train CorEx
topic_model.fit(embeddings, words=words)

# You can save your trained model if you want
topic_model.save('corex_model')

pickle.dump(topic_model, open('corex_model.pkl', 'wb'))

# Get the topics
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

0: 88, 692, 127, 105, 319, 734, 165, 198, 762, 605
1: 728, 10, 591, 100, 428, 286, 527, 122, 23, 316
2: 136, 218, 164, 6, 650, 72, 517, 456, 685, 107
3: 236, 582, 18, 162, 531, 589, 225, 571, 274, 261
4: 550, 731, 625, 177, 594, 467, 147, 233, 244, 189
5: 87, 14, 193, 471, 518, 552, 435, 479, 302, 503
6: 337, 245, 112, 214, 95, 732, 546, 282, 151, 506
7: 158, 672, 365, 161, 355, 206, 680, 155, 249, 1
8: 195, 358, 36, 675, 146, 132, 389, 138, 614, 211
9: 547, 648, 656, 76, 444, 117, 463, 447, 741, 134
10: 0, 266, 469, 700, 294, 39, 252, 398, 584, 541
11: 446, 92, 455, 20, 543, 209, 58, 478, 481, 665
12: 179, 239, 310, 44, 283, 486, 160, 240, 392, 482
13: 50, 226, 12, 445, 86, 426, 565, 596, 278, 213
14: 259, 351, 13, 748, 704, 706, 521, 55, 196, 312
15: 323, 125, 128, 157, 170, 67, 654, 745, 551, 390
16: 148, 234, 102, 321, 169, 70, 513, 599, 79, 554
17: 489, 427, 756, 59, 632, 210, 676, 371, 520, 8
18: 498, 199, 192, 621, 242, 519, 145, 332, 254, 452
19: 705, 568, 270, 442, 695, 301, 5

In [72]:
# Load the model if it's not already in memory
from corextopic import corextopic as ct
# topic_model = pickle.load(open('corex_model.pkl', 'rb'))

# Define the document
document = df['0'][0]

print(document)

# Get the embeddings for the document
document_embeddings = get_document_embeddings(document)

print()
print(document_embeddings)
print()

# Transform with CorEx
topic_distribution = topic_model.transform(document_embeddings)

# Get the top topic for the document
top_topic = np.argmax(topic_distribution[0])
print("The top topic for the document is:", top_topic)

 7 Listen Share Word embedding is one of the most important techniques in natural language processing(NLP), where words are mapped to vectors of real numbers. Word embedding is capable of capturing the meaning of a word in a document, semantic and syntactic similarity, relation with other words. It also has been widely used for recommender systems and text classification. This tutorial will show a brief introduction of genism word2vec model with an example of generating word embedding for the vehicle make model. Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and 

In [75]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Assuming embeddings is a 2D numpy array where each row is the embedding for a specific document
sparse_embeddings = sparse.csr_matrix(embeddings)

# Calculate cosine similarity for all pairs
cosine_similarities = cosine_similarity(sparse_embeddings)

# Get the top 5 articles most similar to the first one
first_article_similarities = cosine_similarities[0]
top_5_similar_articles = np.argsort(first_article_similarities)[-6:-1]  # -6:-1 instead of -5: to exclude the first article itself

In [76]:
top_5_similar_articles

array([ 1260, 10894, 12337,  1711, 13065])

In [97]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Average the embeddings for each article
df['average_embedding'] = df['embeddings'].apply(lambda x: np.mean(x, axis=0))

# Get a matrix of all average embeddings
average_embeddings = np.stack(df['average_embedding'].values)

# Calculate cosine similarity for all pairs
sparse_average_embeddings = sparse.csr_matrix(average_embeddings)
cosine_similarities = cosine_similarity(sparse_average_embeddings)

# Get the top 5 articles most similar to the first one
first_article_similarities = cosine_similarities[4]
top_5_similar_articles = np.argsort(first_article_similarities)[-6:-1]  # -6:-1 instead of -5: to exclude the first article itself

In [98]:
top_5_similar_articles

array([1049,  685,  392, 1296, 1070])

In [100]:
df['0'][1049]

' 15 Listen Share Deep Learning has taken over the majority of fields in solving complex problems, and the geospatial field is no exception. The title of the article interests you and hence, I hope that you are familiar with satellite datasets; for now, Landsat 5 TM. Little knowledge of how Machine Learning (ML) algorithms work, will help you grasp this hands-on tutorial quickly. For those who are unfamiliar with ML concept, in a nutshell, it is establishing the relationship between a few characteristics (features or Xs) of an entity with its other property (value or label or Y) — we provide plenty of examples (labelled data) to the model so that it learns from it and then predicts values/ labels for the new data (unlabelled data). That is enough of theory brush-up for machine learning! The general problem with satellite data: Two or more feature classes (e.g. built-up/ barren/ quarry) in the satellite data can have similar spectral values, which has made the classification a challengi

In [69]:
topic_words,_,_ = zip(*topic_model.get_topics(topic=top_topic))
print('Top topic words: ', ', '.join(topic_words))

Top topic words:  88, 692, 127, 105, 319, 734, 165, 198, 762, 605


In [105]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.8 MB/s eta 0:00:00


In [114]:
import faiss

# Get a matrix of all average embeddings
average_embeddings = np.stack(df['average_embedding'].values).astype('float32')  # Faiss requires float32 type

# Build the index
dimension = average_embeddings.shape[1]  # dimension of the vectors
index = faiss.IndexFlatL2(dimension)
index.add(average_embeddings)

# Now you can use the index for searching the most similar vectors
def search(index, vector, k=5):
    # vector should be a numpy array of shape (1, dimension)
    distances, indices = index.search(vector, k)
    return distances, indices

# Find the 5 most similar articles to the first one
distances, indices = search(index, average_embeddings[0].reshape(1, -1), k=10)
print(indices)

[[   0 1313  667  400  538  539    1  658 1049  653]]


In [113]:
df['0'][1313]

' 4 Listen Share A chatbot is an artificial intelligence software that can simulate a conversation (or a chat) with a user in natural language through messaging applications, websites, mobile apps or through the telephone, etc. These chatbots can be used in various industries for different purposes. We are going to build a simple chatbot using nltk library( Natural Language Toolkit). It is a leading platform for building Python programs to work with human language data. importing necessary libraries. import the dataset into the pandas data frame. The above data contains 1592 data points and two columns context which can be inferred as the query and text response is the response for that query. we can see that there are null values in the dataset if you open the dataset in excel and observe we can find that in our dataset the data is in different clusters i.e, the same type of questions in one place and then followed by next similar kind of questions. Null values are present for the sam